# Code Setup

In [22]:
import numpy as np
import time
import itertools
import inspect
from functools import partial
from typing import Callable, Iterator

from utils.import_puzzle_input import load_input
from utils.import_puzzle_input import split_puzzle_input

from utils.universal_utils import show_func

from utils.utils_puzzle_2023_12 import *

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload modules before executing code
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
puzzle_input = load_input(2023, 12)

loaded puzzle input


In [3]:
puzzle_input_split = split_puzzle_input(puzzle_input)
puzzle_input_split[0:20]

['??????#?#?#?? 2,2,6',
 '#?????????#???? 1,3,2,2',
 '?#??#..#?#???#??#. 5,1,1,4',
 '#??????#???##??#???? 2,1,3,9',
 '??.?????.? 3,1',
 '?.?.?#?##????.?.?# 1,1,3,2,1',
 '#..?????#.???#? 1,1,1,1,4',
 '#?#??????#??????. 1,1,8,1,1',
 '??????.????#?#??. 2,1,6',
 '?#?????.?..??.#??. 1,2,1,2,1,1',
 '??.#?.#???.#??? 1,2,1,1,4',
 '###?.?#??.?#?. 4,4,2',
 '.?...??#??? 1,1',
 '????#.?.???#??#?#?. 2,2,3,5',
 '.??.?#????????? 1,2,1,1,1',
 '?#.????#????.????# 1,6,1,2',
 '?#???.?????### 2,1,3,4',
 '?????#.??# 2,1,3',
 '??????.?##?#??#????? 3,1,12',
 '.?..??.????#?? 1,1,3,1']

In [4]:
# # censored condition records
# pis_ccr = [line.split(" ")[0] for line in puzzle_input_split]

# #puzzle_inout_split check
# pis_check = [line.split(" ")[1] for line in puzzle_input_split]

# puzzle_input_split_ccr, puzzle_input_split_wsr = map(list, zip(*[line.split(" ") for line in puzzle_input_split]))

In [5]:
# # Let's make a function for trimming the beginning and end of lines for "."s
# we use "in {}" speedup as benchmarked to be faster than "== or ==" or "in []"
# !!! 25-06-08: Should be able to speed up more by having indices not materialised 
# !!! 25-06-08: Not actually used in solution 1
show_func(trim_ccr)

def trim_ccr(s: str):
    question_pound_indices = [i for i, char in enumerate(s) if char in {"?", "#"}]
    line_trimmed = s[question_pound_indices[0]:(question_pound_indices[-1]+1)]
    
    return line_trimmed if question_pound_indices else s



In [6]:
# We create a function to do the reformatting of the input
show_func(reformat_pis)

def reformat_pis(puzzle_input_split: list[str]):
    pis_ccr, pis_wsr = map(list, zip(*[line.split(" ") for line in puzzle_input_split]))

    pis_ccr = list(map(trim_ccr, pis_ccr)) # not strictly neccessary for solutions implmented here
    pis_wsr = [[int(num) for num in check.split(",")] for check in pis_wsr]

    return pis_ccr, pis_wsr



In [7]:
pis_ccr, pis_wsr = reformat_pis(puzzle_input_split)

In [8]:
print(pis_ccr[0:20])

['??????#?#?#??', '#?????????#????', '?#??#..#?#???#??#', '#??????#???##??#????', '??.?????.?', '?.?.?#?##????.?.?#', '#..?????#.???#?', '#?#??????#??????', '??????.????#?#??', '?#?????.?..??.#??', '??.#?.#???.#???', '###?.?#??.?#?', '?...??#???', '????#.?.???#??#?#?', '??.?#?????????', '?#.????#????.????#', '?#???.?????###', '?????#.??#', '??????.?##?#??#?????', '?..??.????#??']


In [9]:
pis_wsr[0:20]

[[2, 2, 6],
 [1, 3, 2, 2],
 [5, 1, 1, 4],
 [2, 1, 3, 9],
 [3, 1],
 [1, 1, 3, 2, 1],
 [1, 1, 1, 1, 4],
 [1, 1, 8, 1, 1],
 [2, 1, 6],
 [1, 2, 1, 2, 1, 1],
 [1, 2, 1, 1, 4],
 [4, 4, 2],
 [1, 1],
 [2, 2, 3, 5],
 [1, 2, 1, 1, 1],
 [1, 6, 1, 2],
 [2, 1, 3, 4],
 [2, 1, 3],
 [3, 1, 12],
 [1, 1, 3, 1]]

# Problem Setup

# General thoughts

# Solutions

## Solution 1:

#### Remarks and notation

**R - Record:** 
- A record is a line of the line-split input `puzzle_input_split`  
- Example: `?###???????? 3,2,1`  

**CCR - Censored Condition (of the) Record:**  
- The leftmost part of the record containing the (partially) censored condition of the springs.
- From the puzzle input
- Example: `?###????????`  

**WSR - Working Specification (of the) Record:**  
- The rightmost part of the record containing the cardinality of each continuous arrangement of working springs (uncensored).  
- From the puzzle input
- Example: `3,2,1`  

**PCR: Possible Condition (of the) Record:**  
- A possible uncensoring of the CCR. 
- Generated from the CCR.
- For each R we are looking for the number of (correct) PCRs that have PSRs that correspond to the WSR of the particular R in the input.
- Example: `.###.....##.`

**PSR - Possible (working) Specification Record (of a PCR):** 
- The record specification of a PCR
- Generated from the PCR which is generated from the CCR.
- Example: `3,2` from PCR `.###.....##.`

**KCR - Known (part of the) Condition Record:**
- The uncensored part of a CCR


 

### Explanation of method

### Implementation

#### Part 01:

We create a function to generate an iterator that will generate Cartesian product combinations of `combinants` to generate PCRs.  
The function also takes in a Callable `condition` that might be used to filter amongst the outputted objects of the iterator. We will use this to apply KCR-filters for each R.

In [10]:
# Generate PCRs
show_func(generate_pcr)

def generate_pcr(n: int, combinants: list[str], condition: Callable[[str], bool] = lambda st: True) -> Iterator[str]:
    return (
        ''.join(p) for p in itertools.product(combinants, repeat=n)
        if condition(''.join(p))
    )



In [11]:
# KCR filter function:
show_func(filter_pcr_by_kcr)

def filter_pcr_by_kcr(pcr: str, kcr_filter: dict[int, str]) -> bool:
    return True if all(pcr[i] == kcr_filter[i] for i in kcr_filter) else False



In [12]:
# Let's create a function that generates the kcr filter dictionaries:
show_func(generate_kcr_filter_from_ccr)

def generate_kcr_filter_from_ccr(ccr: str, known_components: set[str] = {".", "#"}) -> dict[int, str]:
    return {i: char for i, char in enumerate(ccr) if char in known_components}



In [13]:
# Let's create a closure function 
show_func(generate_kcr_filter_func_from_ccr)

def generate_kcr_filter_func_from_ccr(ccr: str, known_components: set[str] = {".", "#"}) -> Callable[[str], bool]:
    kcr_filter = generate_kcr_filter_from_ccr(ccr = ccr, known_components=known_components)
    def filter_pcr(pcr: str) -> bool:
        return filter_pcr_by_kcr(pcr = pcr, kcr_filter=kcr_filter)
    return filter_pcr



#### Part 02:

We generate psr from pcrs

In [14]:
show_func(generate_psr_from_pcr)

def generate_psr_from_pcr(pcr: str) -> list[int]:
    return [len(component) for component in pcr.split(".") if component]



#### Part 03:

!!! 25-06-09 skal vi have parametrisering på basis af R, eller med input fra både ccr og wsr

We run over the puzzle_input

In [15]:
show_func(calc_num_pcr_from_ccr)

def calc_num_pcr_from_ccr(ccr: str, wsr: list[int]) -> int:
    filter_func = generate_kcr_filter_func_from_ccr(ccr = ccr)
    len_ccr = len(ccr)
    desired_combinants = ["#", "."]
    return sum(
        1
        for pcr in generate_pcr(n = len_ccr, combinants = desired_combinants, condition = filter_func)
        if generate_psr_from_pcr(pcr = pcr) == wsr
    )



In [16]:
show_func(show_num_pcr_from_ccr)

def show_num_pcr_from_ccr(ccr: str, wsr: list[int]) -> None:
    i = 0
    filter_func = generate_kcr_filter_func_from_ccr(ccr = ccr)
    print(f"ccr: {ccr}, wsr: {wsr}\n")
    for pcr in generate_pcr(len(ccr), ["#", "."], filter_func):
        psr = generate_psr_from_pcr(pcr)
        if psr == wsr:
            print(f"pcr: {pcr}, psr: {psr}")
            i += 1
    print(f"\ntotal number of pcrs: {i}\n")



In [17]:
for k in range(0,10):
    show_num_pcr_from_ccr(pis_ccr[k], pis_wsr[k])

ccr: ??????#?#?#??, wsr: [2, 2, 6]

pcr: ##.##.######., psr: [2, 2, 6]

total number of pcrs: 1

ccr: #?????????#????, wsr: [1, 3, 2, 2]

pcr: #.###.##.##...., psr: [1, 3, 2, 2]
pcr: #.###.##..##..., psr: [1, 3, 2, 2]
pcr: #.###..##.##..., psr: [1, 3, 2, 2]
pcr: #.###....##.##., psr: [1, 3, 2, 2]
pcr: #.###....##..##, psr: [1, 3, 2, 2]
pcr: #.###.....##.##, psr: [1, 3, 2, 2]
pcr: #..###.##.##..., psr: [1, 3, 2, 2]
pcr: #..###...##.##., psr: [1, 3, 2, 2]
pcr: #..###...##..##, psr: [1, 3, 2, 2]
pcr: #..###....##.##, psr: [1, 3, 2, 2]
pcr: #...###..##.##., psr: [1, 3, 2, 2]
pcr: #...###..##..##, psr: [1, 3, 2, 2]
pcr: #...###...##.##, psr: [1, 3, 2, 2]
pcr: #....###.##.##., psr: [1, 3, 2, 2]
pcr: #....###.##..##, psr: [1, 3, 2, 2]
pcr: #....###..##.##, psr: [1, 3, 2, 2]
pcr: #.....###.##.##, psr: [1, 3, 2, 2]

total number of pcrs: 17

ccr: ?#??#..#?#???#??#, wsr: [5, 1, 1, 4]

pcr: #####..#.#...####, psr: [5, 1, 1, 4]

total number of pcrs: 1

ccr: #??????#???##??#????, wsr: [2, 1, 3, 9]

In [69]:
show_func(solve_puzzle)

def solve_puzzle(puzzle_input_split):
    pis_ccr, pis_wsr = reformat_pis(puzzle_input_split)

    nums = [calc_num_pcr_from_ccr(ccr, wsr) for ccr, wsr in zip(pis_ccr, pis_wsr)]
    return sum(nums)



In [51]:
#7490
solve_puzzle(puzzle_input_split)

KeyboardInterrupt: 

### Checking runtime

#### Benchmarking

#### Runtime profiling

In [70]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [71]:
%lprun -f solve_puzzle -f calc_num_pcr_from_ccr -f generate_kcr_filter_func_from_ccr -f generate_kcr_filter_from_ccr -f filter_pcr_by_kcr -f generate_pcr solve_puzzle(puzzle_input_split)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 0.0109183 s
File: c:\Users\VictorZNygaard\Dropbox\other_educational\code_side_projects\aoc\aoc_2023\utils\utils_puzzle_2023_12.py
Function: generate_pcr at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                               return (
    27      1936      36826.0     19.0     33.7          ''.join(p) for p in itertools.product(combinants, repeat=n)
    28       968      72357.0     74.7     66.3          if condition(''.join(p))
    29                                               )
    30                                           
    31                                           # KCR filter function:
    32                                           def filter_pcr_by_kcr(pcr: str, kcr_filter: dict[int, str]) -> bool:
    33                                               return True if all(pcr[i] == kcr_filter[i] for i in kcr_filter) else False

Total time: 768.512 s
File: c:\Users\VictorZNygaard\Dr

In [72]:
len(pis_ccr)

1000

#### Attempt 2:

In [79]:
import time

In [20]:
def run_calc_num_pcr_from_ccr(start_range, end_range, puzzle_input_split):
    pis_ccr, pis_wsr = reformat_pis(puzzle_input_split)
    
    for k in range(start_range, end_range):
        start_time = time.perf_counter()
        num = calc_num_pcr_from_ccr(pis_ccr[k], pis_wsr[k])
        end_time = time.perf_counter()

        print(f"num: {num}, ccr: {pis_ccr[k]}, wsr: {pis_wsr[k]}, time: {end_time - start_time}")

In [81]:
%lprun -f calc_num_pcr_from_ccr -f generate_kcr_filter_func_from_ccr -f generate_kcr_filter_from_ccr -f filter_pcr_by_kcr -f generate_pcr run_calc_num_pcr_from_ccr(0, 100, puzzle_input_split)

ccr: ??????#?#?#??, wsr: [2, 2, 6], num: 1, time: 0.11749090001103468
ccr: #?????????#????, wsr: [1, 3, 2, 2], num: 17, time: 0.4759928999992553
ccr: ?#??#..#?#???#??#, wsr: [5, 1, 1, 4], num: 1, time: 1.4635283999959938
ccr: #??????#???##??#????, wsr: [2, 1, 3, 9], num: 10, time: 11.471964000011212
ccr: ??.?????.?, wsr: [3, 1], num: 4, time: 0.011574000003747642
ccr: ?.?.?#?##????.?.?#, wsr: [1, 1, 3, 2, 1], num: 2, time: 2.5923633000056725
ccr: #..?????#.???#?, wsr: [1, 1, 1, 1, 4], num: 6, time: 0.34486359999573324
ccr: #?#??????#??????, wsr: [1, 1, 8, 1, 1], num: 1, time: 0.7123164999939036
ccr: ??????.????#?#??, wsr: [2, 1, 6], num: 33, time: 0.7109480000071926
ccr: ?#?????.?..??.#??, wsr: [1, 2, 1, 2, 1, 1], num: 4, time: 1.3084422999963863
ccr: ??.#?.#???.#???, wsr: [1, 2, 1, 1, 4], num: 4, time: 0.347790999992867
ccr: ###?.?#??.?#?, wsr: [4, 4, 2], num: 2, time: 0.09563039999920875
ccr: ?...??#???, wsr: [1, 1], num: 4, time: 0.009792600001674145
ccr: ????#.?.???#??#?#?, wsr: [2

Timer unit: 1e-07 s

Total time: 0 s
File: c:\Users\VictorZNygaard\Dropbox\other_educational\code_side_projects\aoc\aoc_2023\utils\utils_puzzle_2023_12.py
Function: generate_pcr at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def generate_pcr(n: int, combinants: list[str], condition: Callable[[str], bool] = lambda st: True) -> Iterator[str]:
    26                                               return (
    27                                                   ''.join(p) for p in itertools.product(combinants, repeat=n)
    28                                                   if condition(''.join(p))
    29                                               )

Total time: 0 s
File: c:\Users\VictorZNygaard\Dropbox\other_educational\code_side_projects\aoc\aoc_2023\utils\utils_puzzle_2023_12.py
Function: filter_pcr_by_kcr at line 32

Line #      Hits         Time  Per Hit   % Time  Line Contents
    32                    

### toy profile 

In [30]:
def level_three():
    total = 0
    for i in range(1000):
        total += i
    return total

def level_two():
    result = level_three()
    return result * 2

def level_one():
    value = level_two()
    return value + 5

def main():
    final = level_one()
    print(f"Final result: {final}")

In [29]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [31]:
%lprun -f main -f level_one -f level_two -f level_three main()

Final result: 999005


Timer unit: 1e-07 s

Total time: 0.0028379 s

Could not find file C:\Users\VictorZNygaard\AppData\Local\Temp\ipykernel_16116\711290020.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1         10.0     10.0      0.0  
     3      1001      14555.0     14.5     51.3  
     4      1000      13745.0     13.7     48.4  
     5         1         69.0     69.0      0.2  

Total time: 0.0041331 s

Could not find file C:\Users\VictorZNygaard\AppData\Local\Temp\ipykernel_16116\711290020.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           
     8         1      41274.0  4

## Solution 2.1: Recursive memoised solution

In [40]:
from functools import lru_cache

def solve_line_lru(ccr: str, wsr: list[int]) -> int:

    @lru_cache(None)
    def count_valid(i: int, block_idx: int, run_len: int) -> int:
        # Base case 1: reached end of string
        if i == len(ccr):
            if run_len == 0: # we’re not inside a partial run
                 return int(block_idx == len(wsr))
            elif run_len == wsr[block_idx]:
                return int(block_idx == len(wsr) - 1)
            else: 
                return 0

        # Determine current character options
        options = [ccr[i]] if ccr[i] != "?" else [".", "#"]

        total = 0
        for ch in options:
            if ch == ".":
                if run_len == 0:
                    # Stay between blocks
                    total += count_valid(i + 1, block_idx, 0)
                else:
                    # End of a block
                    if block_idx < len(wsr) and run_len == wsr[block_idx]:
                        total += count_valid(i + 1, block_idx + 1, 0)
            else:  # ch == "#"
                if block_idx < len(wsr):
                    if run_len < wsr[block_idx]:
                        total += count_valid(i + 1, block_idx, run_len + 1)
                    # No else — placing too many "#"s is invalid

        return total

    return count_valid(0, 0, 0)

In [33]:
for k in range(0, 10):
    print(solve_line_lru(pis_ccr[k], pis_wsr[k]))

1
17
1
10
4
2
6
1
33
4


In [27]:
for k in range(0, 10):
    show_num_pcr_from_ccr(pis_ccr[k], pis_wsr[k])

ccr: ??????#?#?#??, wsr: [2, 2, 6]

pcr: ##.##.######., psr: [2, 2, 6]

total number of pcrs: 1

ccr: #?????????#????, wsr: [1, 3, 2, 2]

pcr: #.###.##.##...., psr: [1, 3, 2, 2]
pcr: #.###.##..##..., psr: [1, 3, 2, 2]
pcr: #.###..##.##..., psr: [1, 3, 2, 2]
pcr: #.###....##.##., psr: [1, 3, 2, 2]
pcr: #.###....##..##, psr: [1, 3, 2, 2]
pcr: #.###.....##.##, psr: [1, 3, 2, 2]
pcr: #..###.##.##..., psr: [1, 3, 2, 2]
pcr: #..###...##.##., psr: [1, 3, 2, 2]
pcr: #..###...##..##, psr: [1, 3, 2, 2]
pcr: #..###....##.##, psr: [1, 3, 2, 2]
pcr: #...###..##.##., psr: [1, 3, 2, 2]
pcr: #...###..##..##, psr: [1, 3, 2, 2]
pcr: #...###...##.##, psr: [1, 3, 2, 2]
pcr: #....###.##.##., psr: [1, 3, 2, 2]
pcr: #....###.##..##, psr: [1, 3, 2, 2]
pcr: #....###..##.##, psr: [1, 3, 2, 2]
pcr: #.....###.##.##, psr: [1, 3, 2, 2]

total number of pcrs: 17

ccr: ?#??#..#?#???#??#, wsr: [5, 1, 1, 4]

pcr: #####..#.#...####, psr: [5, 1, 1, 4]

total number of pcrs: 1

ccr: #??????#???##??#????, wsr: [2, 1, 3, 9]

In [24]:
run_calc_num_pcr_from_ccr(0, 10, puzzle_input_split)

num: 1, ccr: ??????#?#?#??, wsr: [2, 2, 6], time: 0.02162460000545252
num: 17, ccr: #?????????#????, wsr: [1, 3, 2, 2], time: 0.07362290000310168
num: 1, ccr: ?#??#..#?#???#??#, wsr: [5, 1, 1, 4], time: 0.22870439999678638
num: 10, ccr: #??????#???##??#????, wsr: [2, 1, 3, 9], time: 2.272850200009998
num: 4, ccr: ??.?????.?, wsr: [3, 1], time: 0.002879199993913062
num: 2, ccr: ?.?.?#?##????.?.?#, wsr: [1, 1, 3, 2, 1], time: 0.6014010999933816
num: 6, ccr: #..?????#.???#?, wsr: [1, 1, 1, 1, 4], time: 0.06430670000554528
num: 1, ccr: #?#??????#??????, wsr: [1, 1, 8, 1, 1], time: 0.18337939999764785
num: 33, ccr: ??????.????#?#??, wsr: [2, 1, 6], time: 0.19410780000907835
num: 4, ccr: ?#?????.?..??.#??, wsr: [1, 2, 1, 2, 1, 1], time: 0.7715350000071339


In [28]:
show_num_pcr_from_ccr(pis_ccr[2], pis_wsr[2])

ccr: ?#??#..#?#???#??#, wsr: [5, 1, 1, 4]

pcr: #####..#.#...####, psr: [5, 1, 1, 4]

total number of pcrs: 1



In [34]:
solve_line_lru(pis_ccr[2], pis_wsr[2])

1

In [ ]:
nums = [solve_line_lru(pis_ccr[k], pis_wsr[k]) for k in range(0, len(pis_wsr))]

In [39]:
np.sum(nums)

np.int64(7490)

Let's try with manual memoisation

In [ ]:
def solve_line_mm_old(ccr: str, wsr: list[int]) -> int:
    memo = {}

    def count_valid(i: int, block_idx: int, run_len: int) -> int:
        memo_key = (i, block_idx, run_len)
        if memo_key in memo:
            return memo[memo_key]
        
        # Base case 1: reached end of string
        if i == len(ccr):
            if run_len == 0: # we’re not inside a partial run
                 return int(block_idx == len(wsr))
            elif run_len == wsr[block_idx]:
                return int(block_idx == len(wsr) - 1)
            else: 
                return 0

        # Determine current character options
        options = [ccr[i]] if ccr[i] != "?" else [".", "#"]

        total = 0
        for ch in options:
            if ch == ".":
                if run_len == 0:
                    # Stay between blocks
                    total += count_valid(i + 1, block_idx, 0)
                else:
                    # End of a block
                    if block_idx < len(wsr) and run_len == wsr[block_idx]:
                        total += count_valid(i + 1, block_idx + 1, 0)
            else:  # ch == "#"
                if block_idx < len(wsr):
                    if run_len < wsr[block_idx]:
                        total += count_valid(i + 1, block_idx, run_len + 1)
                    # No else — placing too many "#"s is invalid

        memo[memo_key] = total
        return total

    return count_valid(0, 0, 0)

In [42]:
np.sum([solve_line_mm_old(pis_ccr[k], pis_wsr[k]) for k in range(0, len(pis_wsr))])

np.int64(7490)

In [59]:
cache_hits_lst = np.zeros(len(pis_ccr))
cache_misses_lst = np.zeros(len(pis_ccr))

def solve_line_mm_np(ccr: str, wsr: list[int]) -> int:
    memo = {}
    cache_hits = 0
    cache_misses = 0
    
    def count_valid(i: int, block_idx: int, run_len: int) -> int:
        nonlocal cache_hits
        nonlocal cache_misses
     
        memo_key = (i, block_idx, run_len)
        if memo_key in memo:
            cache_hits = cache_hits + 1
            return memo[memo_key]
        cache_misses = cache_misses + 1
        
        # Base case 1: reached end of string
        if i == len(ccr):
            if run_len == 0: # we’re not inside a partial run
                 return int(block_idx == len(wsr))
            elif run_len == wsr[block_idx]:
                return int(block_idx == len(wsr) - 1)
            else: 
                return 0

        # Determine current character options
        options = [ccr[i]] if ccr[i] != "?" else [".", "#"]

        total = 0
        for ch in options:
            if ch == ".":
                if run_len == 0:
                    # Stay between blocks
                    total += count_valid(i + 1, block_idx, 0)
                else:
                    # End of a block
                    if block_idx < len(wsr) and run_len == wsr[block_idx]:
                        total += count_valid(i + 1, block_idx + 1, 0)
            else:  # ch == "#"
                if block_idx < len(wsr):
                    if run_len < wsr[block_idx]:
                        total += count_valid(i + 1, block_idx, run_len + 1)
                    # No else — placing too many "#"s is invalid

        memo[memo_key] = total
        return total
    
    result = count_valid(0, 0, 0)
    np.append(cache_hits_lst, cache_hits)
    np.append(cache_misses_lst, cache_misses)
    return result

In [ ]:
### !!! 31_07_25. Ikke særlig funktionel skrevet med global. Vi bør lave den om
### !!! 31_07_25 Lav et træ-diagram over iterationen ligesom fibonacci

###          F_5
###        /      \
###       F_3       F_4
###      /   \
###     F_1  F_2
###          /  \
###        F_0   F_1
cache_hits_lst = []
cache_misses_lst = []

def solve_line_mm(ccr: str, wsr: list[int]) -> int:
    memo = {}
    cache_hits = 0
    cache_misses = 0
    
    def count_valid(i: int, block_idx: int, run_len: int) -> int:
        nonlocal cache_hits
        nonlocal cache_misses
     
        memo_key = (i, block_idx, run_len)
        if memo_key in memo:
            cache_hits = cache_hits + 1
            return memo[memo_key]
        cache_misses = cache_misses + 1
        
        # Base case 1: reached end of string
        if i == len(ccr):
            if run_len == 0: # we’re not inside a partial run
                 return int(block_idx == len(wsr))
            elif run_len == wsr[block_idx]:
                return int(block_idx == len(wsr) - 1)
            else: 
                return 0

        # Determine current character options
        options = [ccr[i]] if ccr[i] != "?" else [".", "#"]

        total = 0
        for ch in options:
            if ch == ".":
                if run_len == 0:
                    # Stay between blocks
                    total += count_valid(i + 1, block_idx, 0)
                else:
                    # End of a block
                    if block_idx < len(wsr) and run_len == wsr[block_idx]:
                        total += count_valid(i + 1, block_idx + 1, 0)
            else:  # ch == "#"
                if block_idx < len(wsr):
                    if run_len < wsr[block_idx]:
                        total += count_valid(i + 1, block_idx, run_len + 1)
                    # No else — placing too many "#"s is invalid

        memo[memo_key] = total
        return total
    
    result = count_valid(0, 0, 0)
    cache_hits_lst.append(cache_hits)
    cache_misses_lst.append(cache_misses)
    return result

In [61]:
np.sum([solve_line_mm(pis_ccr[k], pis_wsr[k]) for k in range(0, len(pis_ccr))])

np.int64(7490)

In [63]:
np.mean(cache_hits_lst)

np.float64(4.486)

In [64]:
np.mean(cache_misses_lst)

np.float64(54.904)

# Experimental

## Solution 1

In [26]:
pis_ccr

['??????#?#?#??',
 '#?????????#????',
 '?#??#..#?#???#??#',
 '#??????#???##??#????',
 '??.?????.?',
 '?.?.?#?##????.?.?#',
 '#..?????#.???#?',
 '#?#??????#??????',
 '??????.????#?#??',
 '?#?????.?..??.#??',
 '??.#?.#???.#???',
 '###?.?#??.?#?',
 '?...??#???',
 '????#.?.???#??#?#?',
 '??.?#?????????',
 '?#.????#????.????#',
 '?#???.?????###',
 '?????#.??#',
 '??????.?##?#??#?????',
 '?..??.????#??',
 '#.?#?#?..?????.??',
 '???#?.?#?#??',
 '???.??#??.??',
 '?????##?#??????????',
 '??#?????..?##',
 '????.#??.#',
 '????###.???#??.#',
 '??#?.#??.?#?.???',
 '#???#?#...###?##?#',
 '?#??.????',
 '#?#?.???#????####',
 '#?#?.?????##??#????',
 '?####?????.#???',
 '??.??.???##??',
 '#?#??#??..#??##??',
 '????????????',
 '#?????#???.????',
 '?????.????#?#?',
 '?.##???.?#????',
 '?.??##????????',
 '??#?##???##?????#?',
 '?????.#.???#??.?',
 '?????????',
 '#??#?????.#',
 '???.#?.?????',
 '?..?#..??#?',
 '??#????????.???####',
 '??###?#????##?',
 '??#??#???.#????#',
 '#??#?.???#??',
 '?#?##....#?#?#??

In [27]:
# Let's make a function for trimming the beginning and end of lines for "."s
# we use "in {}" speedup as benchmarked to be faster than "== or ==" or "in []"
# !!! 25-06-08: Should be able to speed up more by having indices not materialised 
def trim_ccr(line: str):
    question_pound_indices = [i for i, char in enumerate(line) if char in {"?", "#"}]
    line_trimmed = line[question_pound_indices[0]:(question_pound_indices[-1]+1)]
    
    return line_trimmed if not question_pound_indices else line
    

In [28]:
test_line = "..#.?#?#?..?????.??.."

In [29]:
trim_ccr(test_line)

'..#.?#?#?..?????.??..'

In [30]:
# Let's trim ccr
pis_ccr = list(map(trim_ccr, pis_ccr))

In [31]:
pis_ccr

['??????#?#?#??',
 '#?????????#????',
 '?#??#..#?#???#??#',
 '#??????#???##??#????',
 '??.?????.?',
 '?.?.?#?##????.?.?#',
 '#..?????#.???#?',
 '#?#??????#??????',
 '??????.????#?#??',
 '?#?????.?..??.#??',
 '??.#?.#???.#???',
 '###?.?#??.?#?',
 '?...??#???',
 '????#.?.???#??#?#?',
 '??.?#?????????',
 '?#.????#????.????#',
 '?#???.?????###',
 '?????#.??#',
 '??????.?##?#??#?????',
 '?..??.????#??',
 '#.?#?#?..?????.??',
 '???#?.?#?#??',
 '???.??#??.??',
 '?????##?#??????????',
 '??#?????..?##',
 '????.#??.#',
 '????###.???#??.#',
 '??#?.#??.?#?.???',
 '#???#?#...###?##?#',
 '?#??.????',
 '#?#?.???#????####',
 '#?#?.?????##??#????',
 '?####?????.#???',
 '??.??.???##??',
 '#?#??#??..#??##??',
 '????????????',
 '#?????#???.????',
 '?????.????#?#?',
 '?.##???.?#????',
 '?.??##????????',
 '??#?##???##?????#?',
 '?????.#.???#??.?',
 '?????????',
 '#??#?????.#',
 '???.#?.?????',
 '?..?#..??#?',
 '??#????????.???####',
 '??###?#????##?',
 '??#??#???.#????#',
 '#??#?.???#??',
 '?#?##....#?#?#??

In [32]:
def split_full_stop(line: str) -> list:
    return line.split(".")

In [33]:
split_full_stop

<function __main__.split_full_stop(line: str) -> list>

In [34]:
# let's make a function that generates a list of the number of continuous "#" substrings and their lengths.
def count_pound(line: str) -> list[int] | None:
    line.split()

In [35]:
puzzle_input_split

['??????#?#?#?? 2,2,6',
 '#?????????#???? 1,3,2,2',
 '?#??#..#?#???#??#. 5,1,1,4',
 '#??????#???##??#???? 2,1,3,9',
 '??.?????.? 3,1',
 '?.?.?#?##????.?.?# 1,1,3,2,1',
 '#..?????#.???#? 1,1,1,1,4',
 '#?#??????#??????. 1,1,8,1,1',
 '??????.????#?#??. 2,1,6',
 '?#?????.?..??.#??. 1,2,1,2,1,1',
 '??.#?.#???.#??? 1,2,1,1,4',
 '###?.?#??.?#?. 4,4,2',
 '.?...??#??? 1,1',
 '????#.?.???#??#?#?. 2,2,3,5',
 '.??.?#????????? 1,2,1,1,1',
 '?#.????#????.????# 1,6,1,2',
 '?#???.?????### 2,1,3,4',
 '?????#.??# 2,1,3',
 '??????.?##?#??#????? 3,1,12',
 '.?..??.????#?? 1,1,3,1',
 '#.?#?#?..?????.??.. 1,5,5,1',
 '.???#?.?#?#?? 1,1,4',
 '???.??#??.?? 3,1',
 '?????##?#??????????. 1,6,2,1,1',
 '..??#?????..?## 4,3',
 '????.#??.# 1,1,1',
 '????###.???#??.#. 7,5,1',
 '??#?.#??.?#?.??? 2,2,2,1,1',
 '.#???#?#...###?##?# 1,4,8',
 '.?#??.???? 4,1',
 '#?#?.???#????#### 1,1,4,6',
 '#?#?.?????##??#???? 1,2,8,1,1',
 '?####?????.#??? 5,1,3',
 '??.??.???##?? 1,1,7',
 '#?#??#??..#??##?? 1,6,1,3',
 '???????????? 1,1,1,4'

In [36]:
import itertools
from typing import Callable, Iterator

def generate_combinations(n: int, combinants: list[str], condition: Callable[[str], bool] = lambda x: True) -> Iterator[str]:
    combinants_str = "".join(combinants)
    return (
        ''.join(p) for p in itertools.product(combinants, repeat=n)
        if condition(''.join(p))
    )

In [37]:
# Only combinations ending in "bb"
def ends_with_bb(s: str) -> bool:
    return s.endswith('bb')

for combo in generate_combinations(3, ["a", "b"], ends_with_bb):
    print(combo)

abb
bbb


In [38]:
for combo in generate_combinations(3, ["a", "b"]):
    print(combo)

aaa
aab
aba
abb
baa
bab
bba
bbb


In [39]:
test_list = ["a", "b"]

In [40]:
print("".join(test_list))

ab


### test set experimentation

In [41]:
test_pis_cens_cond_rec = [
    "???.###",
    ".??..??...?##.",
    "?#?#?#?#?#?#?#?",
    "????.#...#...",
    "????.######..#####.",
    "?###????????",
    ]

In [42]:
test_pis_check = [
 "1,1,3",
 "1,1,3",
 "1,3,1,6",
 "4,1,1",
 "1,6,5",
 "3,2,1",
]

In [43]:
test_pis_correct_combinations = [1,4,1,1,4,10]

In [44]:
# let's try to generate all combinations that will satisfy the game conditions for a single example
# censored condition record
test_case_ccr = test_pis_cens_cond_rec[5]
print(f"test_case_ccr: {test_case_ccr}")

# working records specification
test_case_wsr = list(map(int, test_pis_check[5].split(",")))
print(f"test_case_wsr: {test_case_wsr}")

test_case_ccr: ?###????????
test_case_wsr: [3, 2, 1]


In [45]:
def test_filter_index(s: str, test_case_dict: dict) -> bool:
    # True if all(s[i] == test_case_dict[i] for i, char in enumerate(s) if char in ["#", "."]) else False
    # return True if all(char == test_case_dict[i] for i, char in enumerate(s) if char in ["#", "."]) else False
    return True if all(s[i] == test_case_dict[i] for i in test_case_dict) else False

In [ ]:
i = 0
#possible condition record
for pcr in generate_combinations(len(test_case_ccr), [".", "#"]):
    print(pcr)
    i+=1
    if i > 20: # so as not to print too many
        break

............
...........#
..........#.
..........##
.........#..
.........#.#
.........##.
.........###
........#...
........#..#
........#.#.
........#.##
........##..
........##.#
........###.
........####
.......#....
.......#...#
.......#..#.
.......#..##
.......#.#..


In [47]:
from functools import partial
test_case_dict = {1: "#", 2: "#", 3: "#"}
test_case_filter_index = partial(test_filter_index, test_case_dict = test_case_dict)
i = 0
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    print(pcr)
    i += 1
print(i)

.###........
.###.......#
.###......#.
.###......##
.###.....#..
.###.....#.#
.###.....##.
.###.....###
.###....#...
.###....#..#
.###....#.#.
.###....#.##
.###....##..
.###....##.#
.###....###.
.###....####
.###...#....
.###...#...#
.###...#..#.
.###...#..##
.###...#.#..
.###...#.#.#
.###...#.##.
.###...#.###
.###...##...
.###...##..#
.###...##.#.
.###...##.##
.###...###..
.###...###.#
.###...####.
.###...#####
.###..#.....
.###..#....#
.###..#...#.
.###..#...##
.###..#..#..
.###..#..#.#
.###..#..##.
.###..#..###
.###..#.#...
.###..#.#..#
.###..#.#.#.
.###..#.#.##
.###..#.##..
.###..#.##.#
.###..#.###.
.###..#.####
.###..##....
.###..##...#
.###..##..#.
.###..##..##
.###..##.#..
.###..##.#.#
.###..##.##.
.###..##.###
.###..###...
.###..###..#
.###..###.#.
.###..###.##
.###..####..
.###..####.#
.###..#####.
.###..######
.###.#......
.###.#.....#
.###.#....#.
.###.#....##
.###.#...#..
.###.#...#.#
.###.#...##.
.###.#...###
.###.#..#...
.###.#..#..#
.###.#..#.#.
.###.#..#.##
.###.#..##..

In [48]:
# Let's create a function that generates the filter dictionaries:
def generate_filter_dict(s: str, filter: set[str]) -> dict:
    return {i: char for i, char in enumerate(s) if char in filter}

In [49]:
generate_filter_dict(test_case_ccr, {"#", "."})

{1: '#', 2: '#', 3: '#'}

Having had the filter for the uncensored part of the censored condition records (CCR), we need to implement a filter for the working specification (of the) records (WSRs).
We might do this smart somehow, but a dumb way is to simply check each of the possible outputs and whether they match the working specification (of the) records (WSRs) for that record (take intersection).
To do that we might make a function to generate a specification (of a) record  (SR) from a "possible condition record" (PCR).

In [ ]:
def generate_sr_from_pcr(s: str) -> list[int]:
    return [len(component) for component in s.split(".") if component]

In [ ]:
test_case_pcr = ".###....###."
generate_sr_from_pcr(test_case_pcr)

[3, 3]

In [ ]:
i = 0
print(f"ccr: {test_case_ccr}, wsr: {test_case_wsr}")
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    print(f"pcr: {pcr}, sr: {generate_sr_from_pcr(pcr)}")
    i += 1
print(f"total number of pcrs: {i}")

ccr: ?###????????, wsr: [3, 2, 1]
pcr: .###........, sr: [3]
pcr: .###.......#, sr: [3, 1]
pcr: .###......#., sr: [3, 1]
pcr: .###......##, sr: [3, 2]
pcr: .###.....#.., sr: [3, 1]
pcr: .###.....#.#, sr: [3, 1, 1]
pcr: .###.....##., sr: [3, 2]
pcr: .###.....###, sr: [3, 3]
pcr: .###....#..., sr: [3, 1]
pcr: .###....#..#, sr: [3, 1, 1]
pcr: .###....#.#., sr: [3, 1, 1]
pcr: .###....#.##, sr: [3, 1, 2]
pcr: .###....##.., sr: [3, 2]
pcr: .###....##.#, sr: [3, 2, 1]
pcr: .###....###., sr: [3, 3]
pcr: .###....####, sr: [3, 4]
pcr: .###...#...., sr: [3, 1]
pcr: .###...#...#, sr: [3, 1, 1]
pcr: .###...#..#., sr: [3, 1, 1]
pcr: .###...#..##, sr: [3, 1, 2]
pcr: .###...#.#.., sr: [3, 1, 1]
pcr: .###...#.#.#, sr: [3, 1, 1, 1]
pcr: .###...#.##., sr: [3, 1, 2]
pcr: .###...#.###, sr: [3, 1, 3]
pcr: .###...##..., sr: [3, 2]
pcr: .###...##..#, sr: [3, 2, 1]
pcr: .###...##.#., sr: [3, 2, 1]
pcr: .###...##.##, sr: [3, 2, 2]
pcr: .###...###.., sr: [3, 3]
pcr: .###...###.#, sr: [3, 3, 1]
pcr: .###...####.,

We see that we might skip any PCRs that don't have 3 as their first working record number:

In [ ]:
i = 0
print(f"ccr: {test_case_ccr}, wsr: {test_case_wsr}\n")
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    sr = generate_sr_from_pcr(pcr)
    if sr[0] != test_case_wsr[0]:
        continue
    else:
        print(f"pcr: {pcr}, sr: {sr}")
        i += 1
print(f"\ntotal number of pcrs: {i}")

ccr: ?###????????, wsr: [3, 2, 1]

pcr: .###........, sr: [3]
pcr: .###.......#, sr: [3, 1]
pcr: .###......#., sr: [3, 1]
pcr: .###......##, sr: [3, 2]
pcr: .###.....#.., sr: [3, 1]
pcr: .###.....#.#, sr: [3, 1, 1]
pcr: .###.....##., sr: [3, 2]
pcr: .###.....###, sr: [3, 3]
pcr: .###....#..., sr: [3, 1]
pcr: .###....#..#, sr: [3, 1, 1]
pcr: .###....#.#., sr: [3, 1, 1]
pcr: .###....#.##, sr: [3, 1, 2]
pcr: .###....##.., sr: [3, 2]
pcr: .###....##.#, sr: [3, 2, 1]
pcr: .###....###., sr: [3, 3]
pcr: .###....####, sr: [3, 4]
pcr: .###...#...., sr: [3, 1]
pcr: .###...#...#, sr: [3, 1, 1]
pcr: .###...#..#., sr: [3, 1, 1]
pcr: .###...#..##, sr: [3, 1, 2]
pcr: .###...#.#.., sr: [3, 1, 1]
pcr: .###...#.#.#, sr: [3, 1, 1, 1]
pcr: .###...#.##., sr: [3, 1, 2]
pcr: .###...#.###, sr: [3, 1, 3]
pcr: .###...##..., sr: [3, 2]
pcr: .###...##..#, sr: [3, 2, 1]
pcr: .###...##.#., sr: [3, 2, 1]
pcr: .###...##.##, sr: [3, 2, 2]
pcr: .###...###.., sr: [3, 3]
pcr: .###...###.#, sr: [3, 3, 1]
pcr: .###...####.

In [ ]:
# We see that we can find a solution for the test_case
i = 0
print(f"ccr: {test_case_ccr}, wsr: {test_case_wsr}\n")
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    sr = generate_sr_from_pcr(pcr)
    if sr != test_case_wsr:
        continue
    else:
        print(f"pcr: {pcr}, sr: {sr}")
        i += 1
print(f"\ntotal number of pcrs: {i}")

ccr: ?###????????, wsr: [3, 2, 1]

pcr: .###....##.#, sr: [3, 2, 1]
pcr: .###...##..#, sr: [3, 2, 1]
pcr: .###...##.#., sr: [3, 2, 1]
pcr: .###..##...#, sr: [3, 2, 1]
pcr: .###..##..#., sr: [3, 2, 1]
pcr: .###..##.#.., sr: [3, 2, 1]
pcr: .###.##....#, sr: [3, 2, 1]
pcr: .###.##...#., sr: [3, 2, 1]
pcr: .###.##..#.., sr: [3, 2, 1]
pcr: .###.##.#..., sr: [3, 2, 1]

total number of pcrs: 10


We see that we have an approach that will work, but is crude. This will be our solution 1.   
For solution 2 we might try to optimise solution 1.  
In a solution 3, one could optimise the running through of the possibilities by calculating proper PCRs.